In [40]:
import json
import re
from collections import defaultdict
from itertools import combinations
from collections import Counter
from tqdm import tqdm

In [15]:
dataset_name = "muc1700"
muc_1700_input = open("../../Corpora/MUC/muc/processed2/muc_1700_GTT_style-test-dev-train.json")
examples = json.load(muc_1700_input)
muc_1700_input.close()



In [37]:

def clean(string):
    string = string.lower()
    cleaned_string = re.sub(r'[^a-z]', '', string)
    return cleaned_string


def flatten_list(nested_list):
    return [item for sublist in nested_list for item in
            (flatten_list(sublist) if isinstance(sublist, list) else [sublist])]


def n_comb(l):
    return len(l) * (len(l) - 1) // 2

In [61]:
example_template_entity_mentions = []
counts = defaultdict(lambda: 0)
mention_count = []
qualified_mention_count=[]
for ex in tqdm(examples):
    templates = ex['templates']
    text = ex['doctext']
    cleanedtext = clean(" ".join(text))
    template_of_entities = []
    flattened_list_all_templates = []
    for template in templates:
        flattened_list_within_template = []
        arguments = [l for l in template.values() if isinstance(l, list)]
        qualified_entities = []
        per_example_coref_pairs = set()
        per_example_coref_pairs_non_single_mention = set()
        for role_fillers in arguments:
            for entities in role_fillers:
                qualified_mentions = []
                mentions = []
                for mention_w_index in entities:
                    mention = mention_w_index[0]
                    #assert cleanedtext.count(clean(mention)) > 0
                    mentions.append(mention)
                    if cleanedtext.count(clean(mention)) == 1:
                        qualified_mentions.append(mention)
                        counts['qualified_mentions'] += 1

                        if mention in flattened_list_all_templates:
                            counts['duplicated qualified mention across templates'] += 1
                        flattened_list_all_templates.append(mention)

                        if mention in flattened_list_within_template:
                            counts['duplicated qualified mention within templates'] += 1
                        flattened_list_within_template.append(mention)

                if len(qualified_mentions) > 0:
                    counts['qualified_entities'] += 1
                    qualified_entities.append(qualified_mentions)

                    per_example_coref_pairs = per_example_coref_pairs.union(
                        [p for p in list(combinations(qualified_mentions, 2)) if p[0] < p[1]])
                per_example_coref_pairs_non_single_mention = per_example_coref_pairs_non_single_mention.union(
                        [p for p in list(combinations(mentions, 2)) if p[0] < p[1]])
                mention_count.append(len(mentions))
                qualified_mention_count.append(len(qualified_mentions))
        counts['qualified_coenv'] += n_comb(set(flattened_list_within_template))
        counts['actual_coref_pairs'] += len(per_example_coref_pairs)
        counts['actual_coref_pairs_non_single'] += len(per_example_coref_pairs_non_single_mention)
        template_of_entities.append(qualified_entities)

    example_template_entity_mentions.append(template_of_entities)

100%|██████████| 1700/1700 [00:00<00:00, 2232.02it/s]


In [58]:
counts

defaultdict(<function __main__.<lambda>()>,
            {'qualified_coenv': 8541,
             'actual_coref_pairs': 614,
             'actual_coref_pairs_non_single': 1309,
             'qualified_mentions': 4229,
             'qualified_entities': 3269,
             'duplicated qualified mention across templates': 602,
             'duplicated qualified mention within templates': 35})

In [63]:
Counter(mention_count)

Counter({4: 97, 1: 2835, 2: 926, 3: 217, 5: 19, 6: 12, 9: 1, 7: 4})

In [62]:
Counter(qualified_mention_count)

Counter({3: 111, 1: 2549, 2: 553, 0: 842, 4: 42, 5: 11, 6: 3})